In [151]:
# Importing libraries required for handling the data
import pandas as pd
import numpy as np

# Importing libraries for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Importing Principal Component Analysis from sklearn
from sklearn.decomposition import PCA

In [127]:
df = pd.read_csv("https://raw.githubusercontent.com/Brianc482/731_Group_Project/main/Data/Germany.csv")
df.head()

,Name,Age,Nationality,Overall,Potential,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,...,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,PositionClass,Age_Ranges
0,T. Kroos,28,Germany,90,90,76500000.0,355000.0,2190,Right,4.0,5.0,3.0,Medium/ Medium,Normal,Yes,LCM,8.0,"Jul 17, 2014",0,2022,6'0,168lbs,78+3,78+3,78+3,81+3,82+3,82+3,82+3,81+3,84+3,84+3,84+3,82+3,86+3,86+3,86+3,82+3,79+3,82+3,...,72+3,72+3,77+3,88.0,76.0,54.0,92.0,82.0,81.0,86.0,84.0,93.0,90.0,64.0,62.0,70.0,89.0,71.0,87.0,30.0,75.0,73.0,92.0,60.0,82.0,79.0,86.0,73.0,85.0,72.0,79.0,69.0,10.0,11.0,13.0,7.0,10.0,€156.8M,Midfielder,26-30
1,M. ter Stegen,26,Germany,89,92,58000000.0,240000.0,1328,Right,3.0,4.0,1.0,Medium/ Medium,Normal,Yes,GK,22.0,"Jul 1, 2014",0,2022,6'2,187lbs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,15.0,14.0,11.0,36.0,14.0,17.0,18.0,12.0,42.0,18.0,38.0,50.0,37.0,85.0,43.0,22.0,79.0,35.0,79.0,10.0,43.0,22.0,11.0,69.0,25.0,69.0,25.0,13.0,10.0,87.0,85.0,88.0,85.0,90.0,€123.3M,Goalkeeper,26-30
2,M. Neuer,32,Germany,89,89,38000000.0,130000.0,1473,Right,5.0,4.0,1.0,Medium/ Medium,Normal,Yes,GK,1.0,"Jul 1, 2011",0,2021,6'4,203lbs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,15.0,13.0,25.0,55.0,11.0,30.0,14.0,11.0,59.0,48.0,54.0,60.0,51.0,84.0,35.0,25.0,77.0,43.0,80.0,16.0,29.0,30.0,12.0,70.0,47.0,70.0,17.0,10.0,11.0,90.0,86.0,91.0,87.0,87.0,€62.7M,Goalkeeper,31-35
3,M. Hummels,29,Germany,88,88,46000000.0,160000.0,2038,Right,4.0,3.0,3.0,High/ Medium,Normal,Yes,LCB,5.0,"Jul 1, 2016",0,2021,6'3,203lbs,69+3,69+3,69+3,68+3,69+3,69+3,69+3,68+3,71+3,71+3,71+3,70+3,77+3,77+3,77+3,70+3,78+3,83+3,...,85+3,85+3,80+3,64.0,55.0,87.0,81.0,60.0,68.0,65.0,53.0,85.0,81.0,53.0,64.0,63.0,87.0,60.0,71.0,68.0,66.0,84.0,51.0,69.0,92.0,56.0,79.0,68.0,91.0,88.0,90.0,88.0,15.0,6.0,10.0,5.0,6.0,€75.9M,Defender,26-30
4,L. Sané,22,Germany,86,92,61000000.0,195000.0,2090,Left,2.0,3.0,4.0,High/ Medium,Lean,Yes,LW,19.0,"Aug 2, 2016",0,2021,6'0,165lbs,82+2,82+2,82+2,84+2,84+2,84+2,84+2,84+2,82+2,82+2,82+2,83+2,74+2,74+2,74+2,83+2,65+2,59+2,...,52+2,52+2,61+2,83.0,81.0,72.0,79.0,85.0,88.0,82.0,68.0,64.0,85.0,93.0,96.0,88.0,81.0,81.0,86.0,64.0,79.0,70.0,78.0,61.0,34.0,84.0,82.0,71.0,78.0,36.0,32.0,35.0,8.0,12.0,9.0,9.0,14.0,€125.1M,Attacker,22-25


## Data Preparation

Defining a funtion to seperate features which are already in a number format and ready to be used as machine learning inputs 

In [128]:
def numColumns(col):
    t = type(df[col][0])
    if t == np.int64 or t == np.float64:
      print(t,col)
      return col

numberPreds = []

for c in df.columns:
    temp = numColumns(c)
    if temp != None:
        numberPreds.append(temp)

#numberPreds

<class 'numpy.int64'> Age
<class 'numpy.int64'> Overall
<class 'numpy.int64'> Potential
<class 'numpy.float64'> Value
<class 'numpy.float64'> Wage
<class 'numpy.int64'> Special
<class 'numpy.float64'> International Reputation
<class 'numpy.float64'> Weak Foot
<class 'numpy.float64'> Skill Moves
<class 'numpy.float64'> Jersey Number
<class 'numpy.float64'> Crossing
<class 'numpy.float64'> Finishing
<class 'numpy.float64'> HeadingAccuracy
<class 'numpy.float64'> ShortPassing
<class 'numpy.float64'> Volleys
<class 'numpy.float64'> Dribbling
<class 'numpy.float64'> Curve
<class 'numpy.float64'> FKAccuracy
<class 'numpy.float64'> LongPassing
<class 'numpy.float64'> BallControl
<class 'numpy.float64'> Acceleration
<class 'numpy.float64'> SprintSpeed
<class 'numpy.float64'> Agility
<class 'numpy.float64'> Reactions
<class 'numpy.float64'> Balance
<class 'numpy.float64'> ShotPower
<class 'numpy.float64'> Jumping
<class 'numpy.float64'> Stamina
<class 'numpy.float64'> Strength
<class 'numpy.flo

Seperating all the features into different lists

In [ ]:
textCols = ["Name","Nationality","Position","Weak Foot","Preferred Foot", "Work Rate","Release Clause", "PositionClass","Age_Ranges",
            "Body Type","Real Face","Joined", "Loaned From","Contract Valid Until"]

regexCols = [c for c in df.columns if c not in numberPreds] # Keeping columns in string format
regexCols = [c for c in regexCols if c not in textCols] # Removing string features which are not useful/usable

Running Regex on features to replace the '+','-' from various position features, 'lbs' from weight, replacing " ' " with a "." from height. 

In [131]:
df_new = df.copy()

df_new["Height"] = df_new["Height"].replace('\'', '.', regex=True).astype(float)

for c in regexCols:
  df_new[c] = df_new[c].replace('([+|\-|(lbs)].+)|', '', regex=True).astype(float)

predictors = numberPreds + regexCols # Concatinating all the usefull predictors together

encoder = LabelEncoder()
df_new["encodedPosition"]= encoder.fit_transform(df["Position"])

df_new

,Name,Age,Nationality,Overall,Potential,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,...,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,PositionClass,Age_Ranges,encodedPosition
0,T. Kroos,28,Germany,90,90,76500000.0,355000.0,2190,Right,4.0,5.0,3.0,Medium/ Medium,Normal,Yes,LCM,8.0,"Jul 17, 2014",0,2022,6.0,168.0,78.0,78.0,78.0,81.0,82.0,82.0,82.0,81.0,84.0,84.0,84.0,82.0,86.0,86.0,86.0,82.0,79.0,82.0,...,72.0,77.0,88.0,76.0,54.0,92.0,82.0,81.0,86.0,84.0,93.0,90.0,64.0,62.0,70.0,89.0,71.0,87.0,30.0,75.0,73.0,92.0,60.0,82.0,79.0,86.0,73.0,85.0,72.0,79.0,69.0,10.0,11.0,13.0,7.0,10.0,€156.8M,Midfielder,26-30,10
1,M. ter Stegen,26,Germany,89,92,58000000.0,240000.0,1328,Right,3.0,4.0,1.0,Medium/ Medium,Normal,Yes,GK,22.0,"Jul 1, 2014",0,2022,6.2,187.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,15.0,14.0,11.0,36.0,14.0,17.0,18.0,12.0,42.0,18.0,38.0,50.0,37.0,85.0,43.0,22.0,79.0,35.0,79.0,10.0,43.0,22.0,11.0,69.0,25.0,69.0,25.0,13.0,10.0,87.0,85.0,88.0,85.0,90.0,€123.3M,Goalkeeper,26-30,6
2,M. Neuer,32,Germany,89,89,38000000.0,130000.0,1473,Right,5.0,4.0,1.0,Medium/ Medium,Normal,Yes,GK,1.0,"Jul 1, 2011",0,2021,6.4,203.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,15.0,13.0,25.0,55.0,11.0,30.0,14.0,11.0,59.0,48.0,54.0,60.0,51.0,84.0,35.0,25.0,77.0,43.0,80.0,16.0,29.0,30.0,12.0,70.0,47.0,70.0,17.0,10.0,11.0,90.0,86.0,91.0,87.0,87.0,€62.7M,Goalkeeper,31-35,6
3,M. Hummels,29,Germany,88,88,46000000.0,160000.0,2038,Right,4.0,3.0,3.0,High/ Medium,Normal,Yes,LCB,5.0,"Jul 1, 2016",0,2021,6.3,203.0,69.0,69.0,69.0,68.0,69.0,69.0,69.0,68.0,71.0,71.0,71.0,70.0,77.0,77.0,77.0,70.0,78.0,83.0,...,85.0,80.0,64.0,55.0,87.0,81.0,60.0,68.0,65.0,53.0,85.0,81.0,53.0,64.0,63.0,87.0,60.0,71.0,68.0,66.0,84.0,51.0,69.0,92.0,56.0,79.0,68.0,91.0,88.0,90.0,88.0,15.0,6.0,10.0,5.0,6.0,€75.9M,Defender,26-30,9
4,L. Sané,22,Germany,86,92,61000000.0,195000.0,2090,Left,2.0,3.0,4.0,High/ Medium,Lean,Yes,LW,19.0,"Aug 2, 2016",0,2021,6.0,165.0,82.0,82.0,82.0,84.0,84.0,84.0,84.0,84.0,82.0,82.0,82.0,83.0,74.0,74.0,74.0,83.0,65.0,59.0,...,52.0,61.0,83.0,81.0,72.0,79.0,85.0,88.0,82.0,68.0,64.0,85.0,93.0,96.0,88.0,81.0,81.0,86.0,64.0,79.0,70.0,78.0,61.0,34.0,84.0,82.0,71.0,78.0,36.0,32.0,35.0,8.0,12.0,9.0,9.0,14.0,€125.1M,Attacker,22-25,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,J. Hipper,19,Germany,50,59,30000.0,1000.0,806,Right,1.0,2.0,1.0,Medium/ Medium,Normal,No,GK,30.0,"Jul 1, 2017",0,2019,6.0,168.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,13.0,9.0,10.0,21.0,5.0,5.0,11.0,11.0,25.0,11.0,24.0,31.0,31.0,44.0,47.0,17.0,71.0,17.0,51.0,7.0,16.0,12.0,4.0,31.0,11.0,25.0,7.0,10.0,10.0,56.0,47.0,44.0,46.0,51.0,€50K,Goalkeeper,16-21,6
1194,N. Stephan,17,Germany,49,67,50000.0,1000.0,818,Right,1.0,3.0,1.0,Medium/ Medium,Normal,No,GK,24.0,"Jul 1, 2018",0,2020,6.3,185.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,9.0,15.0,28.0,7.0,12.0,13.0,12.0,16.0,10.0,20.0,28.0,21.0,33.0,38.0,21.0,55.0,23.0,58.0,6.0,27.0,9.0,5.0,36.0,15.0,24.0,12.0,13.0,9.0,49.0,48.0,57.0,49.0,54.0,€104K,Goalkeeper,16-21,6
1195,M. Hars

In [130]:
len(predictors)

72

## Principal Component Analysis (PCA)

Since now we have 72 total features that can be used, we used Principal Component analysis to reduce down our feature space and so reducing the training time.

We used PCA to find the least number of components that best retains the information around 95%-99% of the original information.

Scaling the values before running PCA

In [160]:
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df_new[predictors])

Running PCA, with n_components = 0.99. 
This keeps only the tranformed features that retain 99% of the information.

In [161]:
pca = PCA(n_components = 0.99)
reduced = pd.DataFrame(pca.fit_transform(scaled_df))

From 72 features, using PCA we reduced down to 27 transformed features that retains 99% of the information

In [162]:
pca.n_components_

27

In [163]:
reduced

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
0,-2.480846,0.230500,1.263503,-0.220024,0.697485,0.430075,-0.024043,0.392852,-0.052062,-0.068361,0.510270,0.343882,-0.001820,0.064981,0.017457,0.752322,-0.164217,-0.011722,-0.080716,-0.074736,-0.203938,-0.085350,-0.043989,-0.149632,0.045089,-0.006422,-0.078792
1,3.590227,0.144806,1.514229,-0.412965,0.578896,-0.411608,-0.002203,0.180819,0.173892,0.106153,0.190083,0.197486,0.227991,0.072171,-0.066560,0.394958,-0.057170,0.110028,-0.062453,0.033335,-0.062697,0.015001,0.186758,0.014826,0.127645,0.029211,0.000516
2,3.431558,0.238508,1.703955,-0.355218,0.352424,-0.511738,0.120462,0.318730,0.093120,-0.007394,0.135083,-0.063733,0.120047,0.163081,0.117787,0.466506,0.004901,0.265789,-0.013610,-0.010036,0.047157,0.024388,-0.009114,-0.006048,0.046339,-0.157745,-0.131664
3,-1.991467,-0.715591,0.995320,0.019602,0.585682,0.044785,0.229880,0.203358,0.122873,0.106068,0.261112,-0.092052,-0.056967,0.108387,-0.077114,0.440925,-0.124007,0.024346,-0.041535,0.037498,0.005520,-0.015481,-0.000213,-0.042803,0.068412,-0.005056,-0.017884
4,-1.915133,1.184055,0.673529,-0.151987,0.647648,-0.226271,0.215167,-0.140204,-0.005289,0.036715,0.320776,0.068844,0.152625,-0.016770,-0.044086,0.358272,-0.058104,-0.003298,-0.065686,-0.006329,-0.034027,0.010630,-0.049864,0.039020,0.106822,0.006225,0.049525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,3.764775,0.052470,-0.403630,-0.073265,-0.121612,0.233918,-0.034008,-0.030748,0.112639,0.124772,-0.134895,-0.097545,0.029370,0.153444,-0.129268,0.169196,0.006452,-0.065420,-0.077281,0.099357,-0.050391,-0.020716,0.022769,0.052500,0.001111,0.050636,-0.006468
1194,3.753175,0.034212,-0.339627,0.006427,0.124473,0.248042,-0.213540,0.023953,-0.063532,0.049004,-0.078085,-0.021300,0.037565,0.089146,-0.164871,0.081254,-0.013959,0.034400,-0.084315,0.034466,0.011911,0.035661,0.058837,0.037275,0.031772,0.011352,-0.016495
1195,3.739682,0.016717,-0.342760,0.025282,0.041780,0.166302,-0.114960,0.079418,-0.199072,-0.108017,-0.065713,0.074221,0.055428,-0.029252,-0.084222,0.152024,-0.122347,-0.009383,-0.016032,0.048005,0.074189,-0.061946,0.070590,0.048484,-0.026641,-0.004785,-0.008244
1196,0.435487,0.092133,-0.689049,0.011509,-0.203293,-0.019902,-0.125151,-0.081400,0.095412,0.101670,-0.016526,0.069117,-0.174190,0.081286,-0.005302,0.174037,-0.144892,-0.021912,-0.056868,0.043072,0.032761,-0.055515,0.031611,0.034967,0.085863,-0.111230,-0.049927
